# Financial Statements in the OpenBB Platform

OpenBB Platform data extensions provide access to financial statements as quarterly or annual.  There are also endpoints for ratios and other common non-GAAP metrics.  Most data providers require a subscription to access all data. Refer to the website of a specific provider for details on entitlements and coverage.

Financial statement functions are grouped under the `obb.equity.fundamental` module.

## Endpoints

The typical financial statements consist of three endpoints:

- Balance Sheet: `obb.equity.fundamental.balance()`
- Income Statement: `obb.equity.fundamental.income()`
- Cash Flow Statement: `obb.equity.fundamental.cash()`

The main parameters are:

- `symbol`: The company's symbol.
- `period`: 'annual' or 'quarter'. Default is 'annual'.
- `limit`: Limit the number of results returned, from the latest. Default is 5. For perspective, 150 will go back to 1985. The amount of historical records varies by provider.

### Field Names

Some considerations to keep in mind when working with financial statements data are:

- Every data provider has their own way of parsing and organizing the three financial statements.
- Items within each statement will vary by source and by the type of company reporting.
- Names of line items will vary by source.
- "Date" values may differ because they are from the period starting/ending or date of reporting.

This example highlights how different providers will have different labels for compnay facts.

In [ ]:
from openbb import obb
import pandas as pd

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance("TGT", provider="yfinance", limit=4)
  .to_df()["TotalAssets"].reset_index(drop=True)
)

df["fmp"] = (
  obb.equity.fundamental.balance("TGT", provider="fmp", limit=4)
  .to_df()["assets"].convert_dtypes().reset_index(drop=True)
)

df["intrinio"] = (
  obb.equity.fundamental.balance("TGT", provider="intrinio", limit=4)
  .to_df()["total assets"].convert_dtypes().reset_index(drop=True)
)

df["polygon"] = (
  obb.equity.fundamental.balance("TGT", provider="polygon", limit=4)
  .to_df()["total_assets"].convert_dtypes().reset_index(drop=True)
)

df


### Weighted Average Shares Outstanding

This key metric will be found under the income statement.  It might also be called, 'basic', and the numbers do not include authorized but unissued shares.  A declining count over time is a sign that the company is returning capital to shareholders in the form of buy backs.  Under ideal circumstances, it is more capital-efficient, for both company and shareholders, because distributions are double-taxed.  The company pays income tax on paid dividends, and the beneficiary pays income tax again on receipt.

A company will disclose how many shares are outstanding at the end of the period  as a weighted average over the reporting period - three months.

Let's take a look at Target.  To make the numbers easier to read, we'll divide the entire column by one million.

In [ ]:
data = (
  obb.equity.fundamental.income("TGT", provider="fmp", limit=150, period="quarter")
  .to_df()
)

shares = data["weighted_average_shares_outstanding"]/1000000

display(shares.head(1))

display(shares.tail(1))


Thirty-seven years later, the share count is approaching a two-thirds reduction.  12.2% over the past five years.  In four reporting periods, 1.3 million shares have been taken out of the float.

In [ ]:
display(shares.pct_change(20).iloc[-1])

display(shares.iloc[-4] - shares.iloc[-1])


With an average closing price of $143.37, that represents approximately $190M in buy backs.

In [ ]:
price = (
  obb.equity.price.historical("TGT", start_date="2022-10-01", provider="fmp")
  .to_df()
)

round((price["close"].mean()*1300000)/1000000, 2)


### Dividends Paid

Dividends paid is in the cash flow statement.  We can calculate the amount-per-share with the reported data.

In [ ]:
dividends = (
  obb.equity.fundamental.cash("TGT", provider="fmp", limit=150, period="quarter")
  .to_df()[["dividends_paid"]]
)

dividends["shares"] = data["weighted_average_shares_outstanding"]
dividends["div_per_share"] = dividends["dividends_paid"]/dividends["shares"]

dividends["div_per_share"].tail(4)


This can be compared against the real amounts paid to common share holders, as announced.  Note that the dates above represent the report date, and that dividends paid are attributed to the quarter they were paid in.  The value from "2023-01-28" equates to the fourth quarter of 2022.

In [ ]:
(
  obb.equity.fundamental.dividends("TGT", provider="fmp")
  .to_df()["dividend"]
  .loc["2022-11-15":"2023-08-15"]
)


The numbers check out, and the $2B paid to investors over four quarters is more than ten times the $190M returned through share buy backs.

### Financial Attributes

The `openbb-intrinio` data extension has an endpoint for extracting a single fact from financial statements.  There is a helper function for looking up the correct `tag`.

#### Search Financial Attributes

Search attributes by keyword.

In [ ]:
(
    obb.equity.fundamental.search_financial_attributes("marketcap")
    .to_df()
)


The `tag` is what we need, in this case it is what we searched for.

In [ ]:
marketcap = (
  obb.equity.fundamental.financial_attributes(symbol="TGT", tag = "marketcap", period="quarter")
  .to_df()
)

marketcap.tail(5)


Doing some quick math, and ignoring the most recent value, we can see that the market cap of Target was down nearly a quarter over the last four reporting periods.

In [ ]:
(
    (marketcap.loc["2023-09-30"] - marketcap.loc["2022-12-31"])/marketcap.loc["2022-12-31"]
).value


## Ratios and Other Metrics

Other valuation functions are derivatives of the financial statements, but the data provider does the math.  Values are typically ratios between line items, on a per-share basis, or as a percent growth.

This data set is where you can find EPS, FCF, P/B, EBIT, quick ratio, etc.

### Quick Ratio

Target's quick ratio could be one reason why its share price is losing traction against the market.  Its ability to pay current obligations is not optimistically reflected in a 0.27 score, approximately 50% below the historical median.

In [ ]:
ratios = (
  obb.equity.fundamental.ratios("TGT", limit=50, provider="fmp")
  .to_df()
)

display(f"Current Quick Ratio: {round(ratios['quick_ratio'].iloc[-1], 4)}")
display(f"Median Quick Ratio: {round(ratios['quick_ratio'].median(), 4)}")


### Free Cash Flow Yield

The `metrics` endpoint, with the `openbb-fmp` data extension, has a field for free cash flow yield.  It is calculated by taking the free cash flow per share divided by the current share price.  We could arrive at this answer by writing some code, but these types of endpoints do the work so we don't have to.  This is part of the value-add that API data distributors provide, they allow you to get straight to work with data.

We'll use this endpoint to extract the data, and compare with some of Target's competition over the last ten years.

In [ ]:
# List of other retail chains
tickers = ["COST", "BJ", "DLTR", "DG", "WMT", "BIG", "M", "KSS", "TJX"]

# Create a dictionary of tickers and company names.
names = {
    ticker: obb.equity.fundamental.overview(ticker, provider="fmp").results.company_name
    for ticker in tickers
}
# Create a column for each.
fcf_yield = pd.DataFrame()
for ticker in tickers:
    fcf_yield[names[ticker]] = (
        obb.equity.fundamental.metrics(ticker, provider="fmp", period="annual", limit=10)
        .to_df()
        .reset_index()
        .set_index("calendar_year")
        .sort_index(ascending=False)
        ["free_cash_flow_yield"]
    )
fcf_yield.transpose()


There are more usage examples on our [website](https://docs.openbb.co/platform/usage)